In [1]:
import Covid19_model as Cm
import time
from read_training import *
import pickle
from Reinforce_learn import *
import pandas as pd
from Offline_test import *

In [2]:
"""model parameters"""
D = 1e2
n_site = 8
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))/pop
init_state = init_state.tolist()
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
init_state.append([1])
init_state.append([0]*n_site)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1.8e6
path = 'result/transferRL_' + str(round(n_site)) + 'patch'

In [3]:
"""system parameters"""
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
env = Cm.Env_model(init_state, deter_para, in_deter_para, seed=0)
env.set_para_truth(para_truth)
agent = read_training(path, n_state, n_action, env, 54, 54)


model have been loaded


In [5]:
tic = time.time()
state = env.init_state.copy()
observation = state[:6] * pop
state_buffer = np.zeros([T, 6])
new_buffer = np.zeros([T, 4])
budget, score, ni, ne = env.B, 0, 0, 0 
done = False
n_select = 5
epd = 1
p = np.zeros((1, n_site))
action_buffer = []
reward_buffer = []
index = np.array(range(n_select))
actor = agent[0]
start_time = 6
for t in range(T):
    p_set = env.para_sampling(100)
    if t > start_time: 
        agent, select_id = train_online(agent, env, index, epd, p_set)
        actor = agent[select_id]
    state_nn = state[0]+state[1]+state[2]+state[3]+state[4]+state[5]+state[6]+state[7]
    if t > start_time:
        act = actor.act_target(np.array(state_nn))[0]
        action = np.reshape(act, (5, n_site))   
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck, done = env.step_online(t, state, observation,
                                                                             action, p_set,
                                                                             n_select, 100)
    score += reward
    ni += new_i
    ne += new_e
    if not done and t > start_time: 
        env.space_update(ck, p_set, n_select)
        index = ck.argsort()[0][:n_select]
    
    action = np.reshape(action, (5, n_site))
    state = state[:6] * env.pop
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    budget -= cost
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time()-tic)

 Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-114.64
 Time: 0 | S: 18118944.0, E: 56.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1800000.00, reward:-114.64
 Time: 0 | new_e:26.5, cum_e:26.5, new_i:23.6, cum_i:23.6
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 18118899.9, E: 73.9, A: 0.0, I: 23.6, D: 0.0, R: 2.5, Budget: 1800000.00, reward:-277.18
 Time: 1 | S: 18118901.0, E: 72.0, A: 0.0, I: 23.6, D: 0.0, R: 2.0, Budget: 1800000.00, reward:-277.18
 Time: 1 | new_e:37.1, cum_e:63.6, new_i:32.7, cum_i:56.3
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 2 | S: 18118838.0, E: 99.8, A: 0.0, I: 56.3, D: 0.0, R: 5.9, Budget: 1800000.00, reward:-508.20
 Time: 2 | S: 18118841.0, E: 96.0, A: 0.0, I: 56.3, D: 0.0, R: 6.0, Budget: 1800000.00, reward:-508

[0.15669232121309262, 0.15670014946696728]
[0.812867560775292, 0.8128719715989259]
[0.09748681677957265, 0.09748684771444029]
[0.042941693139308824, 0.04294176837010096]
 Time: 13 | S: 17989902.5, E: 20.2, A: 462.7, I: 906.5, D: 3.1, R: 127705.0, Budget: 1632512.91, reward:-5411.25
 Time: 13 | S: 17990002.0, E: 21.0, A: 462.7, I: 906.5, D: 3.1, R: 127690.0, Budget: 1632512.91, reward:-5411.25
 Time: 13 | new_e:9.5, cum_e:1199.8, new_i:58.1, cum_i:1387.7
[[2.4198072e-05 4.6671594e-05 6.1013567e-04 6.8416941e-04 9.9996257e-01
  7.0080715e-03 1.4203953e-02 9.9980849e-01]
 [1.0000000e+00 4.4281472e-04 9.9996269e-01 9.9993694e-01 9.9995804e-01
  9.9995148e-01 1.6233261e-06 9.8820567e-01]
 [9.9998462e-01 9.9999940e-01 9.9999452e-01 9.9993086e-01 9.9995327e-01
  9.9999833e-01 1.0000000e+00 9.9996614e-01]
 [9.9999487e-01 9.9981910e-01 9.9956578e-01 9.9995089e-01 1.0000000e+00
  9.9995840e-01 9.9999869e-01 9.9996376e-01]
 [9.9996531e-01 9.9984980e-01 9.9997640e-01 9.9887651e-01 2.5764036e-01
  

[0.1566951865052564, 0.156695194354026]
[0.8128718670933242, 0.8128718671330829]
[0.0974868445383766, 0.09748684453844575]
[0.0429417643976123, 0.04294176439783572]
 Time: 21 | S: 17840309.5, E: 1.5, A: 139.4, I: 655.1, D: 6.3, R: 277888.3, Budget: 1425492.08, reward:-5666.77
 Time: 21 | S: 17840390.0, E: 2.0, A: 139.4, I: 655.1, D: 6.3, R: 277872.0, Budget: 1425492.08, reward:-5666.77
 Time: 21 | new_e:0.7, cum_e:1218.8, new_i:15.5, cum_i:1623.4
[[9.9991441e-01 7.5837370e-04 9.9539125e-01 9.9369413e-01 1.0000000e+00
  9.6209508e-01 1.6114967e-02 5.1393164e-03]
 [9.9999928e-01 9.9752241e-01 9.9999654e-01 9.9999821e-01 9.9828714e-01
  9.9999833e-01 9.8542249e-01 8.5127169e-01]
 [9.9999964e-01 1.0000000e+00 9.9999964e-01 9.9999988e-01 9.9999976e-01
  9.9999988e-01 9.9999928e-01 1.0000000e+00]
 [1.0000000e+00 9.9999976e-01 9.9999952e-01 9.9999988e-01 1.0000000e+00
  9.9999988e-01 1.0000000e+00 9.9999976e-01]
 [3.2065116e-02 9.9927026e-01 6.0393310e-01 9.2239583e-01 1.2383625e-01
  9.99783

[0.15669518959666373, 0.156695189597029]
[0.8128718671319753, 0.8128718671319799]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822935, 0.042941764397822955]
 Time: 29 | S: 17578670.7, E: 0.4, A: 39.0, I: 328.7, D: 7.4, R: 539953.8, Budget: 1232631.34, reward:-5746.38
 Time: 29 | S: 17578747.0, E: 0.0, A: 39.0, I: 328.7, D: 7.4, R: 539938.0, Budget: 1232631.34, reward:-5746.38
 Time: 29 | new_e:0.2, cum_e:1221.9, new_i:4.3, cum_i:1688.4
[[2.4797827e-02 1.4086121e-03 9.9966335e-01 9.9685007e-01 9.9963272e-01
  1.1906694e-04 2.0773626e-07 1.8489236e-02]
 [2.0681384e-03 9.8060763e-01 6.3602233e-01 9.9814808e-01 9.9982411e-01
  9.9930739e-01 9.9695110e-01 1.9031911e-01]
 [9.9997854e-01 9.9999928e-01 9.9999142e-01 9.9999321e-01 9.9993670e-01
  9.9992394e-01 9.9999118e-01 9.9999833e-01]
 [9.9999964e-01 9.9999452e-01 9.9999297e-01 9.9999487e-01 9.9999619e-01
  9.9995995e-01 9.9999869e-01 9.9986982e-01]
 [9.7175002e-01 9.9999654e-01 9.5361394e-01 9.9607736e-01 1.1299457e-02
  9.999

[0.15669518959679926, 0.15669518959680162]
[0.8128718671319777, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 37 | S: 17449938.0, E: 0.1, A: 10.8, I: 187.5, D: 8.4, R: 668855.2, Budget: 1037758.59, reward:-5809.49
 Time: 37 | S: 17450015.0, E: 0.0, A: 10.8, I: 187.5, D: 8.4, R: 668840.0, Budget: 1037758.59, reward:-5809.49
 Time: 37 | new_e:0.1, cum_e:1222.7, new_i:1.2, cum_i:1706.4
[[2.4977969e-03 4.8889924e-06 9.9964857e-01 9.9718636e-01 4.0189857e-03
  3.2926655e-06 1.1352838e-11 2.4208761e-05]
 [1.5069752e-08 1.9178758e-04 8.0303264e-01 9.9691236e-01 9.9936157e-01
  2.6322423e-02 1.3599493e-01 6.0430635e-04]
 [9.9998403e-01 9.9999905e-01 9.9995220e-01 9.9997520e-01 9.9984086e-01
  9.9997592e-01 9.9999976e-01 9.9998212e-01]
 [9.9999750e-01 9.9999499e-01 9.9999797e-01 9.9995482e-01 9.9993622e-01
  9.9989736e-01 9.9998522e-01 9.9998927e-01]
 [1.7023869e-01 9.9999630e-01 9.8768872e-01 9.3775344e-01 6.4544821e-01
  9.9

...

[0.15669518959679993, 0.15669518959679998]
[0.8128718671319781, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 53 | S: 17325305.9, E: 0.0, A: 0.8, I: 91.0, D: 10.0, R: 793592.3, Budget: 673130.03, reward:-5903.67
 Time: 53 | S: 17325381.0, E: 0.0, A: 0.8, I: 91.0, D: 10.0, R: 793578.0, Budget: 673130.03, reward:-5903.67
 Time: 53 | new_e:0.0, cum_e:1222.9, new_i:0.1, cum_i:1712.8
[[3.6499165e-02 1.8011531e-03 3.3238271e-04 2.2259022e-01 1.1165926e-01
  2.2727362e-04 9.8269582e-01 9.9996245e-01]
 [1.5314126e-05 9.9991059e-01 9.9865216e-01 9.9999750e-01 9.9246514e-01
  2.8117639e-03 9.9998832e-01 9.9999571e-01]
 [9.9999690e-01 1.0000000e+00 9.9999917e-01 9.9999940e-01 9.9999750e-01
  9.9999845e-01 9.9999869e-01 9.9999988e-01]
 [9.9999952e-01 1.0000000e+00 9.9996376e-01 9.9998975e-01 1.0000000e+00
  9.9999881e-01 9.9999976e-01 9.9991000e-01]
 [7.8403306e-01 9.9999988e-01 9.6230841e-01 4.6988293e-03 7.4773121e-01
  9.99998

[0.15669518959679993, 0.15669518959679998]
[0.8128718671319781, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 61 | S: 17285734.1, E: 0.0, A: 0.2, I: 58.4, D: 10.4, R: 833196.8, Budget: 513332.79, reward:-5927.94
 Time: 61 | S: 17285811.0, E: 0.0, A: 0.2, I: 58.4, D: 10.4, R: 833181.0, Budget: 513332.79, reward:-5927.94
 Time: 61 | new_e:0.0, cum_e:1222.9, new_i:0.0, cum_i:1713.1
[[7.9164092e-06 6.4864923e-04 2.3561338e-04 9.9776435e-01 2.7167346e-04
  6.5978238e-05 8.0705130e-01 9.9972099e-01]
 [8.8941044e-05 9.9993694e-01 9.9998462e-01 9.9999988e-01 8.6931139e-01
  9.7274929e-03 9.9999917e-01 9.9998128e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999988e-01 9.9999988e-01
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999046e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [9.2619769e-02 9.9996173e-01 2.0593805e-03 9.5813465e-01 9.5566851e-01
  9.99999

[0.15669518959679993, 0.15669518959679998]
[0.8128718671319781, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 69 | S: 17238652.8, E: 0.0, A: 0.1, I: 46.2, D: 10.8, R: 880290.1, Budget: 379279.96, reward:-5949.66
 Time: 69 | S: 17238728.0, E: 0.0, A: 0.1, I: 46.2, D: 10.8, R: 880276.0, Budget: 379279.96, reward:-5949.66
 Time: 69 | new_e:0.0, cum_e:1222.9, new_i:0.0, cum_i:1713.2
[[1.1043917e-02 8.8513327e-01 2.0605913e-07 1.2712901e-02 5.8644596e-03
  3.1101569e-03 9.9974614e-01 9.4079709e-01]
 [2.7603906e-02 9.9994946e-01 9.5404524e-01 9.9999940e-01 1.2749326e-01
  5.5155724e-01 9.9999964e-01 9.9999833e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 9.9999976e-01 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [6.9035930e-03 9.9377525e-01 1.6050579e-03 5.1148012e-02 7.3484433e-01
  9.99999

[0.15669518959679993, 0.15669518959679998]
[0.8128718671319781, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 77 | S: 17177120.0, E: 0.0, A: 0.0, I: 21.8, D: 10.9, R: 941847.3, Budget: 240243.23, reward:-5956.35
 Time: 77 | S: 17177195.0, E: 0.0, A: 0.0, I: 21.8, D: 10.9, R: 941831.0, Budget: 240243.23, reward:-5956.35
 Time: 77 | new_e:0.0, cum_e:1222.9, new_i:0.0, cum_i:1713.3
[[4.6259566e-04 2.6193645e-03 2.8012165e-08 1.6300371e-05 1.0854997e-01
  8.9751165e-03 9.5196611e-01 9.9920112e-01]
 [6.4022116e-02 1.0000000e+00 9.9991381e-01 9.9997211e-01 9.9585885e-01
  3.1327423e-01 1.0000000e+00 9.9998617e-01]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
  1.0000000e+00 1.0000000e+00 1.0000000e+00]
 [2.6318619e-01 9.9952269e-01 6.6306877e-01 8.4890085e-01 9.3602651e-01
  1.00000

[0.15669518959679993, 0.15669518959679998]
[0.8128718671319781, 0.8128718671319788]
[0.09748684453843849, 0.09748684453843849]
[0.042941764397822955, 0.042941764397822955]
 Time: 85 | S: 17132850.2, E: 0.0, A: 0.0, I: 14.3, D: 11.0, R: 986124.5, Budget: 83403.99, reward:-5961.73
 Time: 85 | S: 17132923.0, E: 0.0, A: 0.0, I: 14.3, D: 11.0, R: 986111.0, Budget: 83403.99, reward:-5961.73
 Time: 85 | new_e:0.0, cum_e:1222.9, new_i:0.0, cum_i:1713.3
[[1.51422660e-06 5.83118663e-06 1.37126106e-08 8.54714036e-01
  2.96533282e-04 2.44028001e-07 4.17965651e-01 9.99975801e-01]
 [9.03881609e-01 9.93955672e-01 9.99994516e-01 9.97633219e-01
  9.99711454e-01 1.19145996e-04 1.00000000e+00 9.99999285e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
 [2.92051900e-02 9.18383360e-01 3.27753602e-04 6.69

In [6]:
(time.time() - tic) / ( T - start_time + 1)

57.14451215126935